# Prelude

In [1]:
from pyiron_workflow import Workflow

In [2]:
from pyironflow import PyironFlow

In [3]:
from pyiron_nodes.atomistic.mlips.fitting.assyst.structures import ElementInput, Elements, ElementsTable

# ASSYST

## Background

- Mg and Defects: https://journals.aps.org/prb/abstract/10.1103/PhysRevB.107.104103
- Ternary Mg/Al/Ca: https://www.researchsquare.com/article/rs-4732459/v1

## Goal

What are we doing here?

## Constructing Element Samplings

In [4]:
mg = Elements((
    {'Mg': 2}, {'Mg': 4}
))
mg

Elements(atoms=({'Mg': 2}, {'Mg': 4}))

In [5]:
al = Elements((
    {'Al': 1}, {'Al': 2}
))
al

Elements(atoms=({'Al': 1}, {'Al': 2}))

Can be combined with standard python operations.

In [6]:
mg + al

Elements(atoms=({'Mg': 2}, {'Mg': 4}, {'Al': 1}, {'Al': 2}))

In [7]:
mg | al

Elements(atoms=({'Mg': 2, 'Al': 1}, {'Mg': 4, 'Al': 2}))

In [14]:
mg * al

Elements(atoms=({'Mg': 2, 'Al': 1}, {'Mg': 2, 'Al': 2}, {'Mg': 4, 'Al': 1}, {'Mg': 4, 'Al': 2}))

Created by the ElementInput node and visualized by StoichiometryTable.

In [9]:
wf = Workflow("ASSYST_Elements_Unary")
wf.Element = ElementInput(element="Mg")
wf.StoichiometryTable = ElementsTable(wf.Element)

In [10]:
pf = PyironFlow([wf])
pf.gui

TASK: Build a small workflow that creates a table with Mg, Al and Ca so that:
1. Mg:Ca is always 2:1
2. combines it with 2-8 Al
3. has at least 2 Ca in every composition
4. contains at most 16 Atoms

Check `utilities` for nodes to `Add()`, `Multiply()` or `Or()` objects together.

Check `atomistic` -> `mlips` -> `fitting` -> `assyst` for nodes to `FilterSize()` or `ElementsTable()`

In [11]:
pf = PyironFlow([])
pf.gui

In [16]:
wf = Workflow("ASSYST_Elements_Combine")

In [ ]:
pf = PyironFlow([wf])
pf.gui

## Full Workflow for a Small Structure Set

In [ ]:
from pyiron_nodes.atomistic.mlips.fitting.assyst import make_assyst

In [ ]:
wf = make_assyst('ASSYST', 'Mg', 'Ca', 'Al', delete_existing_savefiles=True)

In [ ]:
%%time
wf.run()

In [ ]:
pf = PyironFlow([wf], flow_widget_ratio=.85)

HINT available models

In [ ]:
pf.gui

# Precomputed Full Workflow with Large Structure Set